In [ ]:
import os
import cv2
import csv
import random
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score, cross_val_predict, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
%load_ext tensorboard
import datetime

## Preprocess Videos

In [ ]:
def preprocess_frame(frame):
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = tf.image.resize(frame, (224,224))
    #frame = frame / 255.0  # Normalize pixel values
    return frame
    
def preprocess_video(video_path, signer):
    frames = []
    if os.path.exists(video_path):
        cap = cv2.VideoCapture(video_path)

        label = video_path.split("/")[5]
        folder_path = f".../Test/{label}"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        t = 0
        while True:
            ret, cur_frame = cap.read()
            name = video_path.split("_")[9]
            if not os.path.exists(f"{folder_path}/{name}_{t}_{signer}.png"):
                if not ret:
                    break
                frame = preprocess_frame(cur_frame)
                frame = frame.numpy()
                #frames.append(cur_frame)
                cv2.imwrite(f"{folder_path}/{name}_{t}_{signer}.png", frame)
            t += 1
        #fin_frames = stack_images(frames)
        #print(len(fin_frames))
        #for fram in fin_frames:
        #    name = video_path.split("_")[9]
        #    cv2.imwrite(f"{folder_path}/{name}_{t}.png", fram)
        #    t += 1
        cap.release()

    else:
        return 0

In [ ]:
data_dir = ".../KArSL/Signer_1/Train"
data_dir2 = ".../KArSL/Signer_2/Train"
data_dir3 = ".../KArSL/Signer_3/Train"
classes_to_preprocess = ["0160", "0162", "0169", "0173", "0174", "0230", "0231", "0265", "0272", "0290", 
                         "0294", "0296", "0299", "0343", "0459", "0468", "0487", "0497", "0501", "0502"]

for clas in tqdm(os.listdir(data_dir)):
    if clas in classes_to_preprocess:
        for vid in os.listdir(f"{data_dir}/{clas}"):
            preprocess_video(f"{data_dir}/{clas}/{vid}", "1")
            
for clas in tqdm(os.listdir(data_dir2)):
    if clas in classes_to_preprocess:
        for vid in os.listdir(f"{data_dir}/{clas}"):
            preprocess_video(f"{data_dir}/{clas}/{vid}", "2")
            
for clas in tqdm(os.listdir(data_dir3)):
    if clas in classes_to_preprocess:
        for vid in os.listdir(f"{data_dir}/{clas}"):
            preprocess_video(f"{data_dir}/{clas}/{vid}", "3")

## Extract Skleleton Keypoints from Images

In [ ]:
def preprocess_images(images_dir, label, folder_name):
    frames = []
    mp_holistic = mp.solutions.holistic.Holistic(static_image_mode=False)
    for img in os.listdir(f"{images_dir}"):
        frame = cv2.imread(f"{images_dir}/{img}")
        keypoints_file = f".../KArSL/all Signers/{folder_name}_keypoints.csv"
        
        with open(keypoints_file, mode='a', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)

            frame = cv2.resize(frame, (224,224))
            results = mp_holistic.process(image=frame)

            keypoints = []
            if results.left_hand_landmarks:
                for landmarks in results.left_hand_landmarks.landmark:
                    keypoints.extend([landmarks.x, landmarks.y])
            if results.right_hand_landmarks:
                for landmarks in results.right_hand_landmarks.landmark:
                    keypoints.extend([landmarks.x, landmarks.y])

            if len(keypoints) >= 42:  # Ensure at least 43 keypoints are present
                keypoints = keypoints[:42]  # Truncate to 43 keypoints
                keypoints_with_label = [label] + keypoints
                csv_writer.writerow(keypoints_with_label)
                        
            elif keypoints:
                keypoints_with_label = [label] + keypoints
                csv_writer.writerow(keypoints_with_label)
            else:
                #print(f"error with {label}")
                continue

    mp_holistic.close()

In [ ]:
for images_folder in tqdm(os.listdir("D:/WLASL/KArSL/all Signers/Train")):
    preprocess_images(f".../KArSL/all Signers/Test/{images_folder}", images_folder, "train")
for images_folder in tqdm(os.listdir("D:/WLASL/KArSL/all Signers/Test")):
    preprocess_images(f".../KArSL/all Signers/Test/{images_folder}", images_folder, "test")

## Call and Preprocess Skeleton Keypoints

In [ ]:
train_data = pd.read_csv(".../KArSL/all Signers/train_keypoints.csv", header=None)
test_data = pd.read_csv(".../KArSL/all Signers/test_keypoints.csv", header=None)

In [ ]:
X_train = train_data.iloc[:,1:]
y_train = train_data.iloc[:,0]

X_test = test_data.iloc[:,1:]
y_test = test_data.iloc[:,0]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.7, random_state=42)

In [ ]:
labelencoder = LabelEncoder()
labelencoder.fit(y_train)
y_train = labelencoder.transform(y_train)
y_test = labelencoder.transform(y_test)

## Train on RandomForest model

In [ ]:
param_grid = {'n_estimators': [60,80,100,120,140,160,180,200,220,240,260,280], 'max_features': ['sqrt', 'log2']}
grid_search = GridSearchCV(RandomForestClassifier(verbose=2), param_grid)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print("Best Grid: ", best_params)
#plot_grid_search(grid_search.cv_results_, param_grid["n_estimators"], param_grid["max_features"], 'n_estimators', 'max_features')
print("Grid Search is Done")

In [ ]:
rf = RandomForestClassifier(**best_params)
model = rf.fit(X_train,y_train)

## Train on SVM model

In [ ]:
param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [0.1, 1, 10, 100]}
grid_search = GridSearchCV(SVC(probability=True), param_grid)
grid_search.fit(X_train,y_train)
best_params = grid_search.best_params_
print("Best Grid: ", best_params)
print("Grid Search is Done")

In [ ]:
rf = SVC(**best_params, probability=True)
model = rf.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()
print(cm)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred,average='macro')
recall = recall_score(y_test, y_pred, average='macro')
#accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy 1 :", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
#print("Accuracy 2 :", accuracy2)

## Train on CNN-LSTM

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Specify the path to the reorganized dataset folders
train_data_dir = 'D:/WLASL/KArSL/all Signers/Train'
test_data_dir = 'D:/WLASL/KArSL/all Signers/Test'
#test2_data_dir = 'D:/WLASL/KArSL/Signer_2/all_image_test'
# Create an ImageDataGenerator instance with appropriate data augmentation settings
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    validation_split=0.20,
    #rotation_range=0.2,  # Example augmentation settings, modify as needed
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #brightness_range=[0.2,1.0]
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0  # Normalize pixel values
)
# Prepare the training data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=64,
    shuffle=True,
    class_mode='categorical',
    seed=42,
    subset = "training"
)

val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224,224),
    batch_size=64,
    shuffle=True,
    class_mode='categorical',
    seed=42,
    subset = "validation"
)


# Prepare the testing data generator
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224,224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model = tf.keras.models.Sequential([
    #data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.7),
    tf.keras.layers.LSTM(32, dropout=0.7),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(20, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              metrics=["accuracy",
                      tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.AUC(name='AUC')])

model.fit(train_generator, epochs=200, batch_size=64
          , validation_data=val_generator, callbacks=[early_stopping, tensorboard_callback])
%tensorboard --logdir logs/fit

## Train on CNN model

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model = tf.keras.models.Sequential([
    #data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(20, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              metrics=["accuracy",
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall'),
                       tf.keras.metrics.AUC(name='AUC')])

model.fit(train_generator, epochs=200, batch_size=128
          , validation_data=val_generator, callbacks=[early_stopping, tensorboard_callback])
%tensorboard --logdir logs/fit

In [ ]:
model.evaluate(test_generator)

In [ ]:
predictions = model.predict(test_generator)

# Convert predictions and true labels to arrays
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices)
disp.plot()

In [ ]:
model.save('D:/WLASL/KArSL/model_5_cnnlstm.h5')

In [ ]:
model_1 = tf.keras.models.load_model('D:/WLASL/KArSL/model_5_cnnlstm.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('D:/WLASL/KArSL/model_5_cnnlstm.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
model_1.summary()